# PTEN cis effect

This notebook creates a boxplot for the cis effect of PTEN cnv deletions in 8 cancers. These 8 cancers were chosen because they have enough samples with PTEN cnv deletions to do a t-test. Only Ccrcc does not have enough samples with deletions. 

Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import re

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
print('cptac version:', cptac.version())

cptac version: 0.8.5


Functions:

format_df prepares a data frame with PTEN proteomics and PTEN mutation type from get_genotype_all_vars.

format_pval_annotation is used to add marks to the boxplot. A star represents significant p-value and "ns" represents a nonsignificant p-value.

In [3]:
# Returns a dataframe with proteomics and mutation type

def format_df(cancer_object, gene_in, utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    
    if cancer_object.get_cancer_type() not in ('luad'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = gene_in)
        prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
        
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] 
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        del_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    else: 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns=gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1= gene_in, 
            genes2=gene_in)
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = omics.drop(columns='PTEN_CNV')
        # Get only tumor samples
        p = cancer_object.get_proteomics(tissue_type='tumor')
        tumor_ids = list(p.index)
        get = omics.index.isin(tumor_ids)
        omics = omics[get]
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        del_wt = merged[get]
        del_wt['Mutation'] = np.where(
            del_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')

    return del_wt


In [4]:
def format_pval_annotation(pval_symbol, x1, x2, line_start = .05, line_height=.05):
    # for manual adjustment to pval annotations
    
    y, h = line_start, line_height
    plt.plot([x1, x1, x2, x2], #draw horizontal line
             [y, y+h, y+h, y], #vertical line
             lw=1.5, color= '.3')
    plt.text((x1+x2)*.5, # half between x coord
             y+h, pval_symbol, horizontalalignment='center', verticalalignment='bottom', color = "black")


# Step 1: Create data frames with PTEN cnv deletion and Proteomics

Each cancer needs a data frame containing only samples that have PTEN cnv deletions or PTEN wildtype tumors and PTEN proteomics.

First, load in cancer data sets from cptac.

In [5]:
en = cptac.Endometrial()
h = cptac.Hnscc()
l = cptac.Luad()
ls = cptac.Lscc()
o = cptac.Ovarian()
col = cptac.Colon()
g = cptac.Gbm()
b = cptac.Brca()

cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 2)


MissingFileError: Missing data file 'lscc-v3.2-acetylome-ratio-norm-NArm.gct.gz'. Call "cptac.download(dataset='lscc', version='3.2.1')" to download it. Dataset loading aborted.

Next, use format_df to create the specific data frame for each cancer. Use wrap_ttest to test PTEN deletion vs. wildtype. 

# Gbm

In [ ]:
gene = 'PTEN'
g_del_wt = format_df(g,gene)
g_del_wt.head()

In [ ]:
g_pval = u.wrap_ttest(g_del_wt, 'Mutation', ['PTEN_proteomics'])
gp = g_pval.P_Value[0]
g_pval

# Luad

In [ ]:
l_del_wt = format_df(l, gene)
l_pval = u.wrap_ttest(l_del_wt, 'Mutation', ['PTEN_proteomics'])
lp = l_pval.P_Value[0]
l_pval

# Lscc

In [ ]:
ls_del_wt = format_df(ls, gene)
ls_pval = u.wrap_ttest(ls_del_wt, 'Mutation', ['PTEN_proteomics'])
lsp = ls_pval.P_Value[0]
ls_pval

# Ovarian

In [ ]:
o_del_wt = format_df(o, gene)
o_pval = u.wrap_ttest(o_del_wt, 'Mutation', ['PTEN_proteomics'])
op = o_pval.P_Value[0]
o_pval

# Brca

In [ ]:
b_del_wt = format_df(b, gene)
b_pval = u.wrap_ttest(b_del_wt, 'Mutation', ['PTEN_proteomics'])
bp = b_pval.P_Value[0]
b_pval

# Colon

In [ ]:
c_del_wt = format_df(col, gene)
c_pval = u.wrap_ttest(c_del_wt, 'Mutation', ['PTEN_proteomics'])
cp = c_pval.P_Value[0]
c_pval

#  Hnscc

In [ ]:
h_del_wt = format_df(h, gene)
h_pval = u.wrap_ttest(h_del_wt, 'Mutation', ['PTEN_proteomics'])
hp = h_pval.P_Value[0]
h_pval

# Endometrial

The Endometrial data set does not have enough cnv deletions to perform a t-test, however the data set does have enough truncation type mutations (nonsense and frame shifts). Different code is needed to create the data frame for Endometrial.

In [ ]:
# Step 1 - Create proteomics and truncations dataframe
prot_and_mutations = en.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = gene, 
    mutations_filter = ['Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins']) 
keep = ['Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins','Wildtype_Tumor']
in_keep = prot_and_mutations['PTEN_Mutation'].isin(keep)
trunc_mutations = prot_and_mutations[in_keep]
print(trunc_mutations['PTEN_Mutation'].value_counts())

# Step 2 - Create binary column 
trunc_mutations['Mutation'] = np.where(
            trunc_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')

# Step 3 - Format the dataframe correctly for the t-test(just omics and binary columns for tumors)
tumors = trunc_mutations[trunc_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
e_trunc_wt = tumors.drop(columns_to_drop, axis = 1)
e_trunc_wt = e_trunc_wt.dropna(axis=1,how='all')
e_trunc_wt['Mutation'].value_counts()

e_pval = u.wrap_ttest(e_trunc_wt, 'Mutation', ['PTEN_proteomics'])
ep = e_pval.P_Value[0]
e_pval

# Step 2: Create a long dataframe for the boxplot

The boxplot will take three columns: Proteomics, Mutation, and Cancer. We need to append all the individual cancer dfs into one long pancancer df. 

First create the Cancer column.

In [ ]:
g_del_wt = g_del_wt.assign(cancer = 'GBM')
l_del_wt = l_del_wt.assign(cancer = 'LUAD')
ls_del_wt = ls_del_wt.assign(cancer = 'LSCC')
b_del_wt = b_del_wt.assign(cancer = 'BR')
o_del_wt = o_del_wt.assign(cancer = 'OV')
c_del_wt = c_del_wt.assign(cancer = 'CO')
h_del_wt = h_del_wt.assign(cancer = 'HNSCC') # higher scale
e_trunc_wt = e_trunc_wt.assign(cancer = 'EC')

Next append the dfs.

In [ ]:
df = g_del_wt.append(l_del_wt)
df2 = df.append(ls_del_wt)
df3 = df2.append(b_del_wt)
df4 = df3.append(o_del_wt)
df5 = df4.append(c_del_wt)
df6 = df5.append(e_trunc_wt)
#df7 = df6.append(h_del_wt) # Leave out Hnscc because of high proteomics numbers

df6

# Step 3: Create the Pancancer Boxplot

In [ ]:
# cis plot
gene = 'PTEN'
plt.rcParams['figure.figsize']=(11.7, 8.5) #size of plot
sns.set(font_scale = 1.4)

boxplot = sns.boxplot(x='cancer', y=gene+"_proteomics", data = df6, hue = 'Mutation',
                      hue_order = ["Wildtype_Tumor", "Deletion",'Truncation'], showfliers = False)    
#boxplot.set_title('Pancancer cis effect of PTEN CNV Deletions')
boxplot = sns.stripplot(x='cancer', y=gene+"_proteomics", data = df6, jitter = True, 
                           color = ".3", hue = 'Mutation', hue_order = ["Wildtype_Tumor", "Deletion", 'Truncation'], dodge = True)
boxplot.set(xlabel = "\nPTEN Mutation Status", ylabel = gene+' Proteomics')

# format legend
handles, labels = boxplot.get_legend_handles_labels()
plt.legend(handles[0:3], labels[0:3])

# Create significance symbols:
# * P < 0.015   ** P < 0.001   *** P < 0.0001 
cancer_pvals = {'Gbm':gp, 'Hnscc':hp, 'Brca':bp, 'Lscc':lsp, 'Luad':lp, 'Colon':cp, 'Endo':ep, 'Ov':op}

# create pval annotations
symbols = {}
print('p-values: \n')
for cancer in cancer_pvals:
    val = str(cancer_pvals[cancer])
    if cancer_pvals[cancer]  < 0.0001:
        symbols[cancer] = '***'
    elif cancer_pvals[cancer]  < 0.001:
        symbols[cancer] = '**'  
    elif cancer_pvals[cancer]  < 0.015:
        symbols[cancer] = '*'
    print(cancer, val, '\n')
        

format_pval_annotation(symbols['Gbm'], -.3, 0, 1.3) 
format_pval_annotation(symbols['Luad'], .7, 1, 2.2) 
format_pval_annotation(symbols['Lscc'], 1.7, 2, 2.6) # Lscc
format_pval_annotation(symbols['Brca'], 2.7, 3, 2.6) # Brca
format_pval_annotation(symbols['Ov'], 3.7, 4, 1.6) # Ovarian
format_pval_annotation(symbols['Colon'], 4.7, 5, 1)  # Colon
format_pval_annotation(symbols['Endo'], 5.7, 6.2, 1) # Endometrial

plt.show()
plt.clf()
plt.close()

In [ ]:
# Save figure
fig = boxplot.get_figure()
fig.savefig("Sup_Fig_4A.png", dpi = 300)

In [ ]:
# Figure for Hnscc (higher scale)
gene = 'PTEN'
plt.rcParams['figure.figsize']=(1.7,8.5) #size of plot
sns.set(font_scale = 1.4)

boxplot = sns.boxplot(x='cancer', y=gene+"_proteomics", data = h_del_wt, hue = 'Mutation',
                      hue_order = ["Wildtype_Tumor", "Deletion"], showfliers = False)    
boxplot = sns.stripplot(x='cancer', y=gene+"_proteomics", data = h_del_wt, jitter = True, 
                           color = ".3", dodge = True, hue = 'Mutation', hue_order = ["Wildtype_Tumor", "Deletion"],)
boxplot.set(xlabel = "", ylabel = gene+' Proteomics')

# format legend
handles, labels = boxplot.get_legend_handles_labels()
plt.legend(handles[0:0], labels[0:0])

format_pval_annotation(symbols['Hnscc'], -.2, .2, 24.8) 

plt.show()
plt.clf()
plt.close()

In [ ]:
# Save figure
fig = boxplot.get_figure()
fig.savefig("Sup_Fig_4B.png", bbox_inches="tight", dpi = 300)